# Version 1

In [ ]:
from mytoolkit2 import stepwise
import numpy as np

def cal_wqs(input_dir, thres, step):
    files = os.listdir(input_dir)
    wqs = []

    for f in files:
        array= np.loadtxt(os.path.join(input_dir, f))
        #array[array<0] = 0
       
        #array = stepwise.sparsity(array, thres=thres)
        array = stepwise.row_sparsity(array, thres=thres)
      
        wq, *_ = stepwise.findwalks_v2(array, step, normalize=stepwise.min_max)
        wqs.append(wq)
    return wqs

input_dir1 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\ncnc'
input_dir2 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\000d'
input_dir3 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\180d'

wqs_ncnc = cal_wqs(input_dir1, 0.90, 100)
wqs_000d = cal_wqs(input_dir2, 0.90, 100)
wqs_180d = cal_wqs(input_dir3, 0.90, 100)

In [ ]:
seed = #980 
thres = #0.9999
patience = #10

def get_all_stable_cols(wqs, seed, thres, patience):
    stable_cols = []
    for wq in wqs:
        i = stepwise.find_stable_corr(wq, seed, thres, patience, out_dir=None) 
        #find_stable #find_stable_corr
        #i =  4 
        
        stable_col = wq[i][seed]

        #stable_col = stepwise.min_max(stable_col) 

        stable_col = stepwise.z_score(stable_col) 

        stable_cols.append(stable_col)
    return np.array(stable_cols)

stable_ncnc = get_all_stable_cols(wqs_ncnc, seed, thres, patience)
stable_000d = get_all_stable_cols(wqs_000d, seed, thres, patience)
stable_180d = get_all_stable_cols(wqs_180d, seed, thres, patience)


####################################################################################################################
from scipy.stats import ttest_rel
from datasets import mask
import pandas as pd
import numpy as np

m = mask.NiiMask(r"E:\006_ET_MRgFUS\09_gradient_batch2\02_nii_mask\1039atlas.nii")

ts, ps = ttest_rel(stable_180d, stable_000d)
ts_filtered = np.multiply(ts, ps<0.05)
tp = dict(zip(range(1, np.shape(ts)[0]+1), ts_filtered))

print(f'ET180d_vs_ET000d Significant ROI count:{np.sum(ps<0.05)}') 

m.save_values(tp, out_path=r'E:\006_ET_MRgFUS\12_gradient_batch2.1\15_SFC的稳定态度的数值\ET180d_vs_ET000d_tvalue_SFC.nii') 

df = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps})
df.to_csv(r'E:\006_ET_MRgFUS\12_gradient_batch2.1\15_SFC的稳定态度的数值\Part4.1_ET180d_vs_ET000d_SFC.csv') 

###################################################################################################################

ts, ps = ttest_rel(stable_000d, stable_ncnc)
ts_filtered = np.multiply(ts, ps<0.05)
tp = dict(zip(range(1, np.shape(ts)[0]+1), ts_filtered))

print(f'ET180d_vs_ET000d Significant ROI count:{np.sum(ps<0.05)}') 

m.save_values(tp, out_path=r'E:\006_ET_MRgFUS\12_gradient_batch2.1\15_SFC的稳定态度的数值\ET000d_vs_ncncnc_tvalue_SFC.nii') 

df = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps})
df.to_csv(r'E:\006_ET_MRgFUS\12_gradient_batch2.1\15_SFC的稳定态度的数值\Part4.1_ET000d_vs_ncncnc_SFC.csv') 

###################################################################################################################

ts, ps = ttest_rel(stable_180d, stable_ncnc)
ts_filtered = np.multiply(ts, ps<0.05)
tp = dict(zip(range(1, np.shape(ts)[0]+1), ts_filtered))

print(f'ET180d_vs_ET000d Significant ROI count:{np.sum(ps<0.05)}') 

m.save_values(tp, out_path=r'E:\006_ET_MRgFUS\12_gradient_batch2.1\15_SFC的稳定态度的数值\ET180d_vs_ncncnc_tvalue_SFC.nii') 

df = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps})
df.to_csv(r'E:\006_ET_MRgFUS\12_gradient_batch2.1\15_SFC的稳定态度的数值\Part4.1_ET180d_vs_ncncnc_SFC.csv') 


In [ ]:
from scipy.stats import ttest_rel
from datasets import mask
import pandas as pd
import numpy as np

seed = 980 
thres = 0.9999
patience = 10

def get_all_stable_cols(axi, wqs, seed, thres, patience):
    stable_cols = []
    for wq in wqs:
        #i = stepwise.find_stable_corr(wq, seed, thres, patience, out_dir=None) 
        #i =  8
        #find_stable #find_stable_corr
        i=axi

        stable_col = wq[i][seed]

        stable_cols.append(stable_col)
    return np.array(stable_cols)

for i in range(40, 42):
    axi = i

    print("#####################################################")
    print("STEP", axi)
    stable_ncnc = get_all_stable_cols(axi, wqs_ncnc, seed, thres, patience)
    stable_000d = get_all_stable_cols(axi, wqs_000d, seed, thres, patience)
    stable_180d = get_all_stable_cols(axi, wqs_180d, seed, thres, patience)

    m = mask.NiiMask(r"E:\006_ET_MRgFUS\09_gradient_batch2\02_nii_mask\1039atlas.nii")

    ts, ps = ttest_rel(stable_180d, stable_000d)
    ts_filtered = np.multiply(ts, ps<0.0004)
    tp = dict(zip(range(1, np.shape(ts)[0]+1), ts_filtered))
    ts_filtered = np.nan_to_num(ts_filtered)
    print(np.argmax(ts_filtered))
    print(f'ET180d_vs_ET000d Significant ROI count:{np.sum(ps<0.0004)}') 

    ###################################################################################################################

    #ET000d_vs_ncnc脑区的梯度效应值进行统计，并保存
    ts, ps = ttest_rel(stable_000d, stable_ncnc)
    ts_filtered = np.multiply(ts,  ps<0.0004)
    tp = dict(zip(range(1, np.shape(ts)[0]+1), ts_filtered))
    ts_filtered = np.nan_to_num(ts_filtered)
    print(np.argmax(ts_filtered))
    print(f'ET000d_vs_ncnc Significant ROI count:{np.sum(ps<0.0004)}') 

    ###################################################################################################################
    ts, ps = ttest_rel(stable_180d, stable_ncnc)
    ts_filtered = np.multiply(ts,  ps<0.0004)
    tp = dict(zip(range(1, np.shape(ts)[0]+1), ts_filtered))
    ts_filtered = np.nan_to_num(ts_filtered)
    print(np.argmax(ts_filtered))

    print(f'ET180d_vs_ncnc Significant ROI count:{np.sum(ps<0.0004)}') 

In [ ]:
import pandas as pd

seed = 980 
thres = 0.99
patience = 15


def get_all_stable_cols(axi, wqs, seed, thres, patience):
    stable_cols = []
    for wq in wqs:

        i=axi

        stable_col = wq[i][seed]
        
        stable_cols.append(stable_col)
    return np.array(stable_cols)

from scipy.stats import ttest_rel,ttest_ind

for axi in range(0, 100):
    stable_ncnc = get_all_stable_cols(axi, wqs_ncnc, seed, thres, patience)
    stable_000d = get_all_stable_cols(axi, wqs_000d, seed, thres, patience)
    stable_180d = get_all_stable_cols(axi, wqs_180d, seed, thres, patience)

    ts, ps = ttest_rel(stable_ncnc, stable_000d)
    df = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps})
    
    print("###########################")
    print("step", axi+1)  
    print(df)
    df.to_csv(f'E:/006_ET_MRgFUS/12_gradient_batch2.1/step{axi+1}.csv') 

# Version 2

In [ ]:
from mytoolkit2 import stepwise
import numpy as np

def cal_wqs(input_dir, thres, step):
    files = os.listdir(input_dir)
    wqs = []

    for f in files:
        array= np.loadtxt(os.path.join(input_dir, f))
        #array[array<0] = 0
       
        #array = stepwise.sparsity(array, thres=thres)
        array = stepwise.row_sparsity(array, thres=thres)
      
        wq, *_ = stepwise.findwalks_v2(array, step, normalize=stepwise.min_max)
        wqs.append(wq)
    return wqs

input_dir1 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\ncnc'
input_dir2 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\000d'
input_dir3 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\180d'

wqs_ncnc = cal_wqs(input_dir1, 0.90, 100)
wqs_000d = cal_wqs(input_dir2, 0.90, 100)
wqs_180d = cal_wqs(input_dir3, 0.90, 100)

In [ ]:
import pandas as pd

seed = 980 #对应脑区减1
thres = 0.99
patience = 15 #10~20

def get_all_stable_cols(wqs, seed, thres, patience):
    stable_cols = []
    for wq in wqs:
        i = stepwise.find_stable_corr(wq, seed, thres, patience, out_dir=None)
        #i =  8 
        #find_stable #find_stable_corr
        #print(i)
        stable_col = wq[i][seed]

        #stable_col = stepwise.min_max(stable_col) 
        #stable_col = stepwise.z_score(stable_col) 
        
        stable_cols.append(stable_col)
    return np.array(stable_cols)

#print('NNCNC')
stable_ncnc = get_all_stable_cols(wqs_ncnc, seed, thres, patience)
#print('000D')
stable_000d = get_all_stable_cols(wqs_000d, seed, thres, patience)
#print('180D')
stable_180d = get_all_stable_cols(wqs_180d, seed, thres, patience)

#############################################################################################################

from scipy.stats import ttest_rel,ttest_ind
from mytoolkit1.ttest import ind_cohen_d, rel_cohen_d

subnetwork_path = r'E:\006_ET_MRgFUS\09_gradient_batch2\02_nii_mask\7network_atlas1039.csv'
subnetwork_df = pd.read_csv(subnetwork_path)
subnetwork_labels = subnetwork_df['Subnetwork'].values

def get_subnetwork_mean(sub_col, subnetwork_labels):
    means = []
    for j in range(1, 8):
        subnetwork_value = sub_col[subnetwork_labels==j]
        mean = np.nanmean(subnetwork_value)
        means.append(mean)
    return means

def get_subnetwork_means(subs_col, subnetwork_labels):
    subs_means = []
    for sub_col in subs_col:
        means = get_subnetwork_mean(sub_col, subnetwork_labels)
        subs_means.append(means)
    return np.array(subs_means)

subs_means_ncnc = get_subnetwork_means(stable_ncnc, subnetwork_labels)
subs_means_000d = get_subnetwork_means(stable_000d, subnetwork_labels)
subs_means_180d = get_subnetwork_means(stable_180d, subnetwork_labels)

ts, ps = ttest_rel(subs_means_180d, subs_means_000d)
d = rel_cohen_d(subs_means_180d, subs_means_000d)
df1 = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps, 'd_value': d})
print(df1)
#df.to_csv(r'E:\006_ET_MRgFUS\14_SFC_analysis\1.csv') 

ts, ps = ttest_ind(subs_means_000d, subs_means_ncnc)
d = ind_cohen_d(subs_means_000d, subs_means_ncnc)
df2 = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps, 'd_value': d})
print(df2)
#df.to_csv(r'E:\006_ET_MRgFUS\14_SFC_analysis\2.csv') 

ts, ps = ttest_ind(subs_means_180d, subs_means_ncnc)
d = ind_cohen_d(subs_means_180d, subs_means_ncnc)
df3 = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps, 'd_value': d})
print(df3)
#df.to_csv(r'E:\006_ET_MRgFUS\14_SFC_analysis\3.csv') 


In [ ]:

import pandas as pd

seed = 980 
thres = 0.9999 #0.99
patience = 10  #15


def get_all_stable_cols(axi, wqs, seed, thres, patience):
    stable_cols = []
    for wq in wqs:
        #i = stepwise.find_stable_corr(wq, seed, thres, patience, out_dir=None)
        #i =  8 
        #find_stable #find_stable_corr
        i=axi

        stable_col = wq[i][seed]

        ##stable_col = stepwise.min_max(stable_col)
        #stable_col = stepwise.min_max(stable_col) 
        #stable_col = stepwise.z_score(stable_col) 
        
        stable_cols.append(stable_col)
    return np.array(stable_cols)

from scipy.stats import ttest_rel,ttest_ind

subnetwork_path = r'E:\006_ET_MRgFUS\09_gradient_batch2\02_nii_mask\7network_atlas1039.csv'
subnetwork_df = pd.read_csv(subnetwork_path)
subnetwork_labels = subnetwork_df['Subnetwork'].values


def get_subnetwork_mean(sub_col, subnetwork_labels):
    means = []
    for j in range(1, 8):
        subnetwork_value = sub_col[subnetwork_labels==j]
        mean = np.nanmean(subnetwork_value)
        means.append(mean)
    return means

def get_subnetwork_means(subs_col, subnetwork_labels):
    subs_means = []
    for sub_col in subs_col:
        means = get_subnetwork_mean(sub_col, subnetwork_labels)
        subs_means.append(means)
    return np.array(subs_means)

for axi in range(0, 100):
    stable_ncnc = get_all_stable_cols(axi, wqs_ncnc, seed, thres, patience)
    stable_000d = get_all_stable_cols(axi, wqs_000d, seed, thres, patience)
    stable_180d = get_all_stable_cols(axi, wqs_180d, seed, thres, patience)

    subs_means_ncnc = get_subnetwork_means(stable_ncnc, subnetwork_labels)
    subs_means_000d = get_subnetwork_means(stable_000d, subnetwork_labels)
    subs_means_180d = get_subnetwork_means(stable_180d, subnetwork_labels)

    ts, ps = ttest_rel(subs_means_180d, subs_means_000d)
    df = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps})
    
    print("###########################")
    print("step", axi+1)  
    print(df)

